In [2]:
from langchain_qwq import ChatQwen
import os

model = ChatQwen(
    model="qwen-plus",
    api_key=os.getenv("DASHSCOPE_API_KEY"),
    base_url=os.getenv("DASHSCOPE_BASE_URL"),
    temperature=0.7,
)

消息有一个content属性，可以理解为发送给模型的数据有效载荷，该属性是松散类型的（可以接收字符串、列表、字典等格式的数据），这使得LangChain聊天模型能够直接支持特定于提供商的结构，例如多模态内容和其他数据。

# 标准内容块

标准内容块是LangChain提供的一种适用于所有提供商的标准消息内容表示方式。消息对象实现了一个content_blocks属性，该属性会将content属性“惰性解析”为一种标准的、类型安全的表示形式。比如ChatAnthropic 或 ChatOpenAI 生成的消息会包含各自格式的“思考”或“推理”块，但它们可以被惰性解析为一致的ReasoningContentBlock表示形式。

>“惰性解析”（Lazy Parsing）是一种“按需计算”的策略。简单来说，就是“不到万不得已，绝不干活”。模型返回的数据最初可能是一个复杂的JSON字符串或字典，但系统不会立即将其转换成LangChain内部的标准对象，只有当你真正去调用 message.content_blocks 这个属性时，系统才会开始执行转换代码，把原始数据解析成标准格式。

In [4]:
from langchain.messages import AIMessage

message = AIMessage(
    content=[
        {
            "type": "reasoning",
            "id": "rs_abc123",
            "summary": [
                {"type": "summary_text", "text": "summary 1"},
                {"type": "summary_text", "text": "summary 2"},
            ],
        },
        {"type": "text", "text": "...", "id": "msg_abc123"},
    ],
    response_metadata={"model_provider": "openai"}
)
message.content_blocks

[{'type': 'reasoning', 'id': 'rs_abc123', 'reasoning': 'summary 1'},
 {'type': 'reasoning', 'id': 'rs_abc123', 'reasoning': 'summary 2'},
 {'type': 'text', 'text': '...', 'id': 'msg_abc123'}]

如果将LC_OUTPUT_VERSION环境变量设置为v1，或者在初始化任何聊天模型时，将其output_version参数设置为 "v1"，content与content_blocks属性将返回相同的结果，langchain自动在后台完成了这个转化。

In [5]:
model = ChatQwen(
    model="qwen-plus",
    api_key=os.getenv("DASHSCOPE_API_KEY"),
    base_url=os.getenv("DASHSCOPE_BASE_URL"),
    temperature=0.7,
    output_version="v1",
)

In [ ]:
from langchain.messages import HumanMessage
response = model.invoke([HumanMessage(content="介绍一下你自己")])

print("content:", response.content)
print("="*60)
print("content_blocks:", response.content_blocks)

## 核心

### TextContentBlock
标准文本输出

- type (类型)
   -  string (字符串) | 必填
   - 值始终为 "text" (文本)
- text (文本)
   - string (字符串) | 必填
   - 具体的文本内容
- annotations (注释)
   - object[] (对象数组) | 可选
   - 文本的注释列表
- extras (额外数据)
    - object (对象) | 可选
    - 其他特定于提供商的额外数据

示例：
```json
{
    "type": "text",
    "text": "Hello world",
    "annotations": []
}
```

### ReasoningContentBlock
模型的推理步骤

- type (类型)
    - string (字符串) | 必填
    - 值始终为 "reasoning" (推理)
- reasoning (推理内容)
    - string (字符串) | 可选
    - 具体的推理内容
- extras (额外数据)
    - object (对象) | 可选
    - 其他特定于提供商的额外数据

示例：
```json
{
    "type": "reasoning",
    "reasoning": "用户正在询问关于...",
    "extras": {"signature": "abc123"}
}
```

## 多模态

图片、视频、文件、音频结构基本一致，都包含type、url或者base64、id、mime_type。

- type (类型)
    - string (字符串) | 必填
    - 值为"image" 、"video" 、"file" 、"audio"
- 内容数据
    - url或者base64二选一
- id
    - 给这个文件块一个身份证号（通常用于引用或去重）。
- mime_type
    - 说明文件的具体格式（是 jpg 还是 png？是 mp3 还是 wav？）。如果使用了base64，通常必须提供这个，否则程序看不懂编码后的数据是什么格式。
- extras (额外数据)
    - object (对象) | 可选
    - 其他特定于提供商的额外数据

示例：
```json
{
    "type": "image",
    "url": "https://example.com/image.jpg",
    "mime_type": "image/jpeg"
}
```

## 工具调用

### ToolCall
函数调用

- type (类型)
    - string (字符串) | 必填
    - 值始终为 "tool_call"
- name (名称)
    - string (字符串) | 必填
    - 要调用的工具（函数）的名称
- args (参数)
    - object (对象) | 必填
    - 传递给工具的参数
- id (ID)
    - string (字符串) | 必填
    - 该工具调用的唯一标识符

示例：
```json
{
    "type": "tool_call",
    "name": "search",
    "args": {"query": "weather"},
    "id": "call_123"
}
```

### ToolCallChunk
流式传输工具调用片段（用于流式输出场景）

- type (类型)
    - string (字符串) | 必填
    - 值始终为 "tool_call_chunk"
- name (名称)
    - string (字符串) | 可选
    - 正在调用的工具名称
- args (参数片段)
    - string (字符串) | 可选
    - 部分工具参数（可能是不完整的 JSON 字符串）
- id (ID)
    - string (字符串) | 可选
    - 工具调用标识符
- index (索引)
    - number | string (数字或字符串) | 可选
    - 该片段在流中的位置

### InvalidToolCall
格式错误的调用，用于捕获 JSON 解析错误。比较常见在流式响应的工具调用中间过程中，因为此时参数不完整，无法解析，LangChain会将这些中间片段标记为invalid_tool_calls

- type (类型)
    - string (字符串) | 必填
    - 值始终为 "invalid_tool_call"
- name (名称)
    - string (字符串) | 可选
    - 调用失败的工具名称
- args (参数)
    - object (对象) | 可选
    - 传递给工具的参数
- error (错误)
    - string (字符串) | 可选
    - 对发生问题的描述

# 多模态内容
不同的供应商对多模态内容有不同的要求，具体请查看对应的文档。

## 图片消息

In [9]:
# From URL
message = {
    "role": "user",
    "content": [
        {"type": "text", "text": "Describe the content of this image."},
        {"type": "image", "url": "https://example.com/path/to/image.jpg"},
    ]
}

# From base64 data
message = {
    "role": "user",
    "content": [
        {"type": "text", "text": "Describe the content of this image."},
        {
            "type": "image",
            "base64": "AAAAIGZ0eXBtcDQyAAAAAGlzb21tcDQyAAACAGlzb2...",
            "mime_type": "image/jpeg",
        },
    ]
}

## PDF消息

In [10]:
# From URL
message = {
    "role": "user",
    "content": [
        {"type": "text", "text": "Describe the content of this document."},
        {"type": "file", "url": "https://example.com/path/to/document.pdf"},
    ]
}

# From base64 data
message = {
    "role": "user",
    "content": [
        {"type": "text", "text": "Describe the content of this document."},
        {
            "type": "file",
            "base64": "AAAAIGZ0eXBtcDQyAAAAAGlzb21tcDQyAAACAGlzb2...",
            "mime_type": "application/pdf",
        },
    ]
}

## 音频消息

In [12]:
# From base64 data
message = {
    "role": "user",
    "content": [
        {"type": "text", "text": "Describe the content of this audio."},
        {
            "type": "audio",
            "base64": "AAAAIGZ0eXBtcDQyAAAAAGlzb21tcDQyAAACAGlzb2...",
            "mime_type": "audio/wav",
        },
    ]
}

## 视频消息

In [11]:
# From base64 data
message = {
    "role": "user",
    "content": [
        {"type": "text", "text": "Describe the content of this video."},
        {
            "type": "video",
            "base64": "AAAAIGZ0eXBtcDQyAAAAAGlzb21tcDQyAAACAGlzb2...",
            "mime_type": "video/mp4",
        },
    ]
}

此外，还有通过file_id、media_id等id字段传递的，这个根据你使用的模型及其配套服务来决定。